In [ ]:
import numpy as np
import xarray as xr
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.multioutput import MultiOutputRegressor
import GPy
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import xarray as xr
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.multioutput import MultiOutputRegressor

# Load NetCDF data using xarray
file_path = 'path'
data = xr.open_dataset(file_path)

# Extract variables (assuming 'ws' is wind speed and 'tp' is total precipitation)
ws10 = data['ws'].values  # Shape: (time, latitude, longitude)
tp = data['tp'].values  # Shape: (time, latitude, longitude)

# Normalize the data, ignoring NaNs
ws10_mean = np.nanmean(ws10)
ws10_std = np.nanstd(ws10)
tp_mean = np.nanmean(tp)
tp_std = np.nanstd(tp)

normalized_ws10 = (ws10 - ws10_mean) / ws10_std
normalized_tp = (tp - tp_mean) / tp_std

# Prepare the input (X) and output (Y) data
time_indices, lat_indices, lon_indices = np.meshgrid(
    np.arange(normalized_ws10.shape[0]),
    np.arange(normalized_ws10.shape[1]),
    np.arange(normalized_ws10.shape[2]),
    indexing='ij'
)

X = np.vstack([time_indices.ravel(), lat_indices.ravel(), lon_indices.ravel()]).T
Y = np.vstack([normalized_ws10.ravel(), normalized_tp.ravel()]).T

# Filter out NaNs
valid_mask = ~np.isnan(Y).any(axis=1) & ~np.isnan(X).any(axis=1)
X_valid = X[valid_mask]
Y_valid = Y[valid_mask]

# Perform train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X_valid, Y_valid, test_size=0.1, random_state=42)

### Independent Single-Output GPs ###
kernel = RBF()
gpr = GaussianProcessRegressor(kernel=kernel)

# Using MultiOutputRegressor to handle independent outputs
sogp_model = MultiOutputRegressor(gpr)
sogp_model.fit(X_train, Y_train)

# Make predictions with SOGP
Y_pred_sogp = sogp_model.predict(X_test)

# Evaluate SOGP performance
mse_sogp = ((Y_test - Y_pred_sogp) ** 2).mean(axis=0)

### Correlated Multi-Output GPs ###

X_train_ws10 = np.hstack([X_train, np.zeros((X_train.shape[0], 1))])
X_train_tp = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_train_combined = np.vstack([X_train_ws10, X_train_tp])

Y_train_combined = np.hstack([Y_train[:, 0], Y_train[:, 1]])

# Train a single GP on the combined data
kernel_combined = GPy.util.multioutput.LCM(input_dim=3, num_outputs=2, kernels_list=[GPy.kern.RBF(3), GPy.kern.RBF(3)])
gpr_combined = GPy.models.GPCoregionalizedRegression([X_combined], [Y_combined], kernel=kernel)
#gpr_combined.fit(X_train_combined, Y_train_combined)
gpr_combined.optimize()


# Prepare the test data with an extra dimension for output indexing
X_test_ws10 = np.hstack([X_test, np.zeros((X_test.shape[0], 1))])
X_test_tp = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
X_test_combined = np.vstack([X_test_ws10, X_test_tp])

# Make predictions
Y_pred_combined = gpr_combined.predict_noiseless(X_test_combined)

# Separate the predictions
Y_pred_ws10_combined = Y_pred_combined[:X_test.shape[0]]
Y_pred_tp_combined = Y_pred_combined[X_test.shape[0]:]

# Combine the predictions into the original shape
Y_pred_mogp = np.vstack([Y_pred_ws10_combined, Y_pred_tp_combined]).T

from sklearn.metrics import mean_squared_error, meansolute_error, meansolute_percentage_error

# For Single Output Gaussian Processes (SOGPs)
mse_sogp = mean_squared_error(Y_test, Y_pred_sogp, multioutput='raw_values')
mae_sogp = meansolute_percentage_error(Y_test, Y_pred_sogp, multioutput='raw_values')
print("MSE for independent SOGPs:", mse_sogp)
print("MAPE for independent SOGPs:", mae_sogp)

# For Multi-Output Gaussian Processes (MOGPs)
mse_mogp = mean_squared_error(Y_test, Y_pred_mogp, multioutput='raw_values')
mae_mogp = meansolute_percentage_error(Y_test, Y_pred_mogp, multioutput='raw_values')
print("MSE for correlated MOGP:", mse_mogp)
print("MAPE for correlated MOGP:", mae_mogp)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 定义纬度和经度的索引
lat_indices = [5, 6]
lon_indices = [50, 49]
# 提取经纬度数据
latitudes = data['latitude'].values  # 你的纬度坐标名可能不同，根据实际情况调整
longitudes = data['longitude'].values  # 你的经度坐标名可能不同，根据实际情况调整
# 查找对应的经纬度
lat_values = [latitudes[idx] for idx in lat_indices]
lon_values = [longitudes[idx] for idx in lon_indices]

# 创建3x3的图表
fig, axes = plt.subplots(2, 2, figsize=(15, 6))

# Extract time series for the chosen point
def extract_time_series(data, lat_index, lon_index):
    return data[:, lat_index, lon_index]

# 遍历所有纬度和经度的组合
for i, lat in enumerate(lat_indices):
    for j, lon in enumerate(lon_indices):

        # 为每个组合提取时间序列
        ws10_series = extract_time_series(normalized_ws10, lat, lon)
        tp_series = extract_time_series(normalized_tp, lat, lon)

        # Predicted values for the chosen point from SOGP
        X_test_point = np.array([[i, lat, lon] for i in range(normalized_ws10.shape[0])])
        Y_pred_point_sogp = sogp_model.predict(X_test_point)
        Y_pred_point_ws10_sogp = Y_pred_point_sogp[:, 0]
        Y_pred_point_tp_sogp = Y_pred_point_sogp[:, 1]

        # Predicted values for the chosen point from MOGP
        X_test_point_ws10 = np.hstack([X_test_point, np.zeros((X_test_point.shape[0], 1))])
        X_test_point_tp = np.hstack([X_test_point, np.ones((X_test_point.shape[0], 1))])
        X_test_point_combined = np.vstack([X_test_point_ws10, X_test_point_tp])

        Y_pred_point_combined = gpr_combined.predict_noiseless(X_test_point_combined)
        Y_pred_point_ws10_mogp = Y_pred_point_combined[:X_test_point.shape[0]]
        Y_pred_point_tp_mogp = Y_pred_point_combined[X_test_point.shape[0]:]


        # 剪裁时间序列
        time_range = np.arange(ws10_series.shape[0])
        
        
        
        # 绘制 ws10 和 tp
        ax = axes[i, j]
        # 假设这里是预测值的绘制
        ax.plot(time_range, ws10_series, 'b-',label='True ws10')
        ax.plot(time_range, Y_pred_point_ws10_sogp,n,'r--', label='Predicted ws10 (SOGP)')
        ax.plot(time_range, Y_pred_point_ws10_mogp,'g--', label='Predicted ws10 (MOGP)')

        ax.set_xlabel('Time')
        ax.set_ylabel('Normalized ws10')
        ax.set_title(f'Wind Speed at ({lat_values[i]}N, {lon_values[j]}W) For Hurricane XXX(Name)')
        ax.legend()

# 调整布局以避免重叠
plt.tight_layout()
plt.show()
